In [1]:
%matplotlib inline

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

In [2]:
class ARIMA():
    def __init__(self, start_date, month_number, data, randomseed=42):
        self.randomseed = randomseed
        self.alphas = np.linspace(0, 3, 50, dtype=float)
        self.betas = np.linspace(0, 3, 50, dtype=float)
        
        self.start_date = start_date
        self.month_number = month_number
        self.data = data
        
        self.start_index = self.data[self.data.date == self.start_date].index[0]
        
        self.mean = self.data.res.mean()
        self.std = self.data.res.std()
        
        self.x = np.zeros(self.month_number)
        self.eps = np.zeros(self.month_number)
                 
        self.time_interval = self.data.loc[self.start_index:self.start_index+ self.month_number-1].date.values
        self.fact_future = self.data.loc[self.start_index:self.start_index+ self.month_number-1].value_filt.values
        self.fact_past = self.data.loc[self.start_index- self.month_number+1:self.start_index].value.values
        
        
    def arima(self,alpha,beta,month_number):
        np.random.seed(self.randomseed)
        self.eps = np.random.normal(alpha*self.mean, beta*self.std, size=self.month_number)
        self.x[0] = self.data.loc[self.start_index - month_number].value_filt + self.eps[0]
        for i in range (1,self.month_number):
            self.x[i] = self.eps[i] + self.x[i-1]
        return self.x
    
    def error_1(self, alpha,beta, month_number):
        np.random.seed(self.randomseed)
        if month_number == 0:
            err_1 = np.sum(abs(self.fact_future - self.arima(alpha, beta,month_number)))/self.month_number
        else:
            err_1 = np.sum(abs(self.fact_past - self.arima(alpha, beta,month_number)))/self.month_number
        return err_1
    
    def error_2(self,alpha,beta, month_number):
        np.random.seed(self.randomseed)
        if month_number == 0:
            err_2 = abs(np.min(self.fact_future - self.arima(alpha, beta, month_number)))
        else:
            err_2 = abs(np.min(self.fact_past - self.arima(alpha, beta, month_number)))
        return err_2

In [3]:
class OptimizedARIMA(ARIMA):
    
    def optimize(self):
        coeff = []
        errors = []
        month_number = self.month_number
        for alpha in self.alphas:
            for beta in self.betas:
                coeff.append([alpha, beta])
                errors.append(self.error_1(alpha, beta, month_number)+self.error_2(alpha, beta, month_number))
        
        optimum = coeff[np.argmin(errors)]
        alpha_opt = optimum[0]
        beta_opt = optimum[1]
        return np.round(alpha_opt,2), np.round(beta_opt,2)
    
    def forecast_plot(self):
        try:
            f, ax = plt.subplots(figsize=(18, 8))
            ax.plot(self.time_interval, self.arima(self.optimize()[0], self.optimize()[1],0), c='r')
            ax.plot(self.time_interval, self.fact_future, c='g')
 #           ax.set_ylim([10000,75000])
            ax.legend(['Прогноз остатков по методу ARIMA, млн. рублей', 'Фактические остатки, млн. руб.'])
            ax.grid()
            print('Среднемесячное отклонение от фактических остатков:',
                  np.round(self.error_1(self.optimize()[0], self.optimize()[1],0)*100*self.month_number/np.sum(self.fact_future),1),'%')
            print('Максимум пробития млн. руб.:', np.round(self.error_2(self.optimize()[0], self.optimize()[1],0),1))
            print('Оптимальная средняя скорость роста остатков:', np.round(self.optimize()[0]*self.mean,1), 'млн. руб./мес.')
            print('Оптимальное стандартное отклонение остатков:', np.round(self.optimize()[1]*self.std,1), 'млн. руб.')
        except:
            print('ошибка ввода даты или количества прогнозных месяцев')
    
    def forecast_errors(self):
        print('Среднемесячное отклонение от фактических остатков:',
                  np.round(self.error_1(self.optimize()[0], self.optimize()[1],0)*100*self.month_number/np.sum(self.fact_future),1),'%')
        print('Максимум пробития млн. руб.:', np.round(self.error_2(self.optimize()[0], self.optimize()[1],0),1))

In [4]:
def ML_forecast(start_date, forecast_date, data, model):
    
    data['time_step'] = data.index
    
    start_date = start_date
    forecast_date = forecast_date
    
    forecast_days_number = data[data.date == forecast_date].index[0] - data[data.date == start_date].index[0]
    training_range_days = data[data.date == start_date].index[0]
    forecast_time_interval = [training_range_days, training_range_days + forecast_days_number]
    
    X = data.time_step.values.reshape(-1,1)
    y = data.value.values.reshape(-1,1)
    
    X_train = X[:training_range_days]
    X_test = X[training_range_days:training_range_days + forecast_days_number]
    Y_train = y[:training_range_days]
    Y_test = y[training_range_days:training_range_days + forecast_days_number]
    
    Model = model
    Model.fit(X_train,np.ravel(Y_train)) 
    y_pred = Model.predict(X_test)
    
    R_2 = metrics.r2_score(Y_test, y_pred)
    MAE = metrics.mean_absolute_error(Y_test, y_pred)
    error_1 = np.sum(abs(Y_test.T - y_pred.T))*100/len(Y_test)/np.mean(Y_test)
    error_2 = abs(np.min(np.where((Y_test.T - y_pred.T)>0, 0, Y_test.T - y_pred.T)))
    print('Среднемесячное отклонение от фактических остатков:', np.round(error_1,2),'%')
    print('Максимум пробития млн. руб.:', np.round(error_2),'млн. руб.')
    
    f, ax = plt.subplots(figsize=(18, 8))

    ax.plot(data.date[forecast_time_interval[0]:forecast_time_interval[1]], y_pred, c='r')
    ax.plot(data.date[forecast_time_interval[0]:forecast_time_interval[1]], Y_test, c='g')
    #ax.set_ylim([10000,80000])
    ax.legend(['Прогноз остатков, млн. рублей', 'Фактические остатки, млн. руб.'])
    ax.grid()

In [5]:
def ML_forecast_samples(start_date, forecast_date, sample_size, data, model):
    
    start_date = start_date
    forecast_date = forecast_date
    sample_size = sample_size
    
       
    forecast_days_number = data[data.date == forecast_date].index[0] - data[data.date == start_date].index[0]
    training_range_days = data[data.date == start_date].index[0] - sample_size
    forecast_time_interval = [sample_size + training_range_days, sample_size + training_range_days + forecast_days_number]
    
    vals = data.value.values.reshape(-1,1)
    
    X = vals[:sample_size].T
    for i in range(1,training_range_days):
        X = np.vstack([X,vals[i:i+sample_size].T])

    y = vals[sample_size + forecast_days_number].T
    for i in range(1,training_range_days):
        y = np.vstack([y,vals[i+sample_size + forecast_days_number].T])
        
    X_train = X[:training_range_days - forecast_days_number]
    X_test = X[training_range_days - forecast_days_number:training_range_days]
    y_train = y[:training_range_days - forecast_days_number]
    y_test = y[training_range_days - forecast_days_number:training_range_days]
    
    Model = model
    Model.fit(X_train,np.ravel(y_train))
    y_pred = Model.predict(X_test)
    
    return y_pred, y_test

In [6]:
def ML_forecast_optimized_samples(start_date, forecast_date, data, model, max_sample_size=365, min_sample_size = 2):
    
    sample_sizes = np.linspace(min_sample_size,max_sample_size, 20, dtype='int')
    
    error = []
    
    for sample_size in sample_sizes:
        y_pred = ML_forecast_samples(start_date, forecast_date, sample_size, data, model)[0]
        y_test = ML_forecast_samples(start_date, forecast_date, sample_size, data, model)[1]
        error_1 = np.sum(abs(y_test.T - y_pred.T))*100/len(y_test)/np.mean(y_test.T)
        error_2 = abs(np.min(np.where((y_test.T - y_pred.T)>0, 0, y_test.T - y_pred.T)))
        error.append(error_1 + error_2)
    
    sample_size_optimum = sample_sizes[np.argmin(error)]
    y_pred_optimum = ML_forecast_samples(start_date, forecast_date, sample_size_optimum, data, model)[0]
    
    error_1 = np.sum(abs(y_test.T - y_pred_optimum.T))*100/len(y_test)/np.mean(y_test.T)
    error_2 = abs(np.min(np.where((y_test.T - y_pred_optimum.T)>0, 0, y_test.T - y_pred_optimum.T)))
    print('Оптимальный размер сэмпла:', sample_size_optimum)
    print('Среднемесячное отклонение от фактических остатков:', np.round(error_1,2),'%')
    print('Максимум пробития млн. руб.:', np.round(error_2),'млн. руб.')
    
    forecast_time_interval = [data[data.date == start_date].index[0], data[data.date == forecast_date].index[0]]
    
    f, ax = plt.subplots(figsize=(18, 8))

    ax.plot(data.date[forecast_time_interval[0]:forecast_time_interval[1]], y_pred_optimum, c='r')
    ax.plot(data.date[forecast_time_interval[0]:forecast_time_interval[1]], np.ravel(y_test), c='g')
    #ax.set_ylim([10000,80000])
    ax.legend(['Прогноз остатков, млн. рублей', 'Фактические остатки, млн. руб.'])
    ax.grid()  
    

In [28]:
def ML_forecast_features(start_date, forecast_date, sample_size, data, model):
    
    start_date = start_date
    forecast_date = forecast_date
    sample_size = sample_size
    
       
    forecast_days_number = data[data.date == forecast_date].index[0] - data[data.date == start_date].index[0]
    training_range_days = data[data.date == start_date].index[0] - sample_size
    forecast_time_interval = [sample_size + training_range_days, sample_size + training_range_days + forecast_days_number]
    
    vals = data.value.values.reshape(-1,1)
    
    X = vals[:sample_size].T
    for i in range(1,training_range_days):
        X = np.vstack([X,vals[i:i+sample_size].T])

    y = vals[sample_size + forecast_days_number].T
    for i in range(1,training_range_days):
        y = np.vstack([y,vals[i+sample_size + forecast_days_number].T])
    
    df = pd.DataFrame(data = X)
    df['id'] = y
    
    df['sum'] = df.iloc[:, 1:sample_size].sum(axis=1)
    df['min'] = df.iloc[:, 1:sample_size].min(axis=1)
    df['max'] = df.iloc[:, 1:sample_size].max(axis=1)
    df['mean'] = df.iloc[:, 1:sample_size].min(axis=1)
    df['std'] = df.iloc[:, 1:sample_size].std(axis=1)
    df['skew'] = df.iloc[:, 1:sample_size].skew(axis=1)
    df['kurt'] = df.iloc[:, 1:sample_size].kurt(axis=1)
    df['res'] = (df.iloc[:, 1:sample_size].max(axis=1) - df.iloc[:, 1:sample_size].min(axis=1))/(sample_size-1)
    
    df = df[['id','min', 'skew','kurt','sum','max','mean','std','res']]
        
    X_train = df[:training_range_days - forecast_days_number].drop('id', axis=1).values
    X_test = df[training_range_days - forecast_days_number:training_range_days].drop('id', axis=1).values
    y_train = df['id'][:training_range_days - forecast_days_number].values
    y_test = df['id'][training_range_days - forecast_days_number:training_range_days].values
    
    Model = model
    Model.fit(X_train,np.ravel(y_train))
    y_pred = Model.predict(X_test)
    
    return y_pred, y_test

In [8]:
def ML_forecast_optimized_features(start_date, forecast_date, data, model, max_sample_size=365, min_sample_size = 2):
    
    sample_sizes = np.linspace(min_sample_size,max_sample_size, 20, dtype='int')
    
    error = []
    
    for sample_size in sample_sizes:
        y_pred = ML_forecast_features(start_date, forecast_date, sample_size, data, model)[0]
        y_test = ML_forecast_features(start_date, forecast_date, sample_size, data, model)[1]
        error_1 = np.sum(abs(y_test.T - y_pred.T))*100/len(y_test)/np.mean(y_test.T)
        error_2 = abs(np.min(np.where((y_test.T - y_pred.T)>0, 0, y_test.T - y_pred.T)))
        error.append(error_1 + error_2)
    
    sample_size_optimum = sample_sizes[np.argmin(error)]
    y_pred_optimum = ML_forecast_features(start_date, forecast_date, sample_size_optimum, data, model)[0]
    
    error_1 = np.sum(abs(y_test.T - y_pred_optimum.T))*100/len(y_test)/np.mean(y_test.T)
    error_2 = abs(np.min(np.where((y_test.T - y_pred_optimum.T)>0, 0, y_test.T - y_pred_optimum.T)))
    print('Оптимальный размер сэмпла:', sample_size_optimum)
    print('Среднемесячное отклонение от фактических остатков:', np.round(error_1,2),'%')
    print('Максимум пробития млн. руб.:', np.round(error_2),'млн. руб.')
    
    forecast_time_interval = [data[data.date == start_date].index[0], data[data.date == forecast_date].index[0]]
    
    f, ax = plt.subplots(figsize=(18, 8))

    ax.plot(data.date[forecast_time_interval[0]:forecast_time_interval[1]], y_pred_optimum, c='r')
    ax.plot(data.date[forecast_time_interval[0]:forecast_time_interval[1]], np.ravel(y_test), c='g')
    #ax.set_ylim([10000,80000])
    ax.legend(['Прогноз остатков, млн. рублей', 'Фактические остатки, млн. руб.'])
    ax.grid()  